# Ordenando un toque:
    - Lo que buscamos es encontrar algún (o algunos) modelos que nos permitan aproximar alguna de las columnas del dataset (si, alguna).
    - Si o si tenemos que hacerlo co nla columna "precio", pero además tenemos que elgir otra (está en el enunciado).
    
    - Separemos los tantos:
        - Feature Engeniere es algo que vamos a hacer POR FUERA de todo lo que es la ejecución, es decir, cuando nosotros hagamos fit + predict + loquesea, esta nueva columna no tiene idea de que es nueva, asi que esto no nos importa tanto (para lo que es la ejecucion pura y dura)
       
        - Segmentación + K-fold + errores, es algo que nos interesa DENTRO de la ejecución.
        - Ejecutar 1 vez y tomar metricas, no estaría bueno, por lo que ya sabemos (overfitting), entonces no nos queda otra más que para cada segmento, realizar el KFold, callcular las métricas en cada Fold, y guardar el promedio de esas métricas (nos queda un array de metricas promedio en cada segmento).
        - ¿Donde empieza el prolema? En que para cada Fold, los valores de los factores de la ecuación de la recta (esos que obtenemos cuando hacemos fit) van a cambiar siempre, porque para cada "cacho" de dataset, estas cambian.
        - ¿Nos importa realmente? Obviamente googlie esto porque estaba medio perdido, y la respuesta es que no (a priori). Lo primero que a nosotros nos interesa saber, es si nuestro modelo "esta bueno", es decir, si el error total es bajo (comparando distintas métricas y demas). Esto no nos da las variables "finales", una vez que definimos que algún modelo funciona bien, recién ahí ejecutamos la regresión con TODOS los datos, y no quedamos con los factores resultantes (si usamos segmentación, seria un conjunto de factores para cada segmento, y para predecír deberíamos saber determinar a que segmento pertenece)
        
        
Esto lo saque de acá:
https://stats.stackexchange.com/questions/52274/how-to-choose-a-predictive-model-after-k-fold-cross-validation
https://stats.stackexchange.com/questions/2306/feature-selection-for-final-model-when-performing-cross-validation-in-machine?rq=1

Por si estoy entendiendo cualquier cosa.

In [1]:
import metnum
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error as RMSE, mean_squared_log_error as RMSLE

/home/diego/.pyenv/versions/3.6.5/envs/metnum-tp3/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
df_original = pd.read_csv('../data/train.csv')
df = df_original.copy()

In [6]:
class Segment:
    def __init__(self, name, linear_regressor, features):
        self.name = name
        self.linear_regressor = linear_regressor
        self.features = features
        
    def fit(self, A, reales):
        self.linear_regressor.fit(A, reales)

    def predict(self, A):
        return self.linear_regressor.predict(A)
    
    def execute(self, A, reales, metrics, k=5):

        self.errors = np.zeros_like(metrics)

        kf = KFold(n_splits=k)
        for train_index, test_index in kf.split(A):
            # Divido en datos de entrenamiento y testeo
            A_train, A_test = A[train_index], A[test_index]
            reales_train, reales_test = reales[train_index], reales[test_index]
            # Fiteo con los de entrenamiento
            self.fit(A_train, reales_train)

            # Predigo con los de testeo
            aproximados = self.predict(A_test)
            i = 0
            # Calculo y guardo las metricas pasadas por parametro
            for metric in metrics:
                self.errors[i] += metric(reales_test, aproximados)
                i += 1
        
        self.errors /= k


In [7]:
def regresionPorSegmento(df, segment_column, features, metrics, predict_column='precio'):
    segments = np.array([])
    
    for name in df[segment_column].unique():
        # Leo los datos correspondientes al segmeto
        df_segment = df[df[segment_column] == name] # Aquellos datos que no contengan la columna, no son tomados en cuenta
        
        A = df_segment[features].values
        precios_reales = df_segment[predict_column].values

        # Creo el segmento
        segment = Segment(name, metnum.LinearRegression(), features)
        #print(f'Segemento: {name} \n')
        # Fit y predict
        segment.execute(A, precios_reales, metrics)
        
        # Guardo el segmento
        segments = np.append(segments, segment)

    return segments

In [8]:
def exec_model(df, segment, features, metrics):
    # Limpia los valores con NAN que se vayan a utilizar
    df = df.dropna(subset=(features + [segment]))
    
    return regresionPorSegmento(df, segment, features, metrics)

In [9]:
# Modelo fruta
df = df_original.copy()
segment = 'provincia'
features = ['metrostotales', "metroscubiertos"]   #'metrostotales', 'metroscubiertos', "habitaciones", "banos", "antiguedad"]

metrics = [RMSE, RMSLE]

segments = exec_model(df, segment, features, metrics)

In [10]:
for s in segments:
    print(s.errors)

[4156749071464.4873 0.3071691409779902]
[1355194492588.362 0.2932669986485194]
[1830346961682.852 0.3667594151433021]
[740824638139.3896 0.1949426722241246]
[1429756211704.571 0.2684862729582663]
[388832510605.67365 0.11303979967999667]
[408027099872.0533 0.20656126463355434]
[2507226433761.599 0.5654948689918251]
[932832375466.7946 0.25033169218691]
[458740053142.37335 0.14034028903283677]
[551829348110.3738 0.23983246108862893]
[3091724327733.6313 0.45568317097932176]
[1151826759557.8696 0.31166457496731514]
[961830726823.1814 0.25869121846451754]
[467217023201.6852 0.10813658750774782]
[660078633755.4427 0.28600566646616815]
[452110401394.4484 0.20387552327436023]
[884162547358.2389 0.5432835474034555]
[508304407194.47833 0.17915311217726956]
[424213926038.9456 0.20798367519430414]
[242345793981.44858 0.2610792431345274]
[332634478184.8405 0.22302118536357432]
[155052789398.07208 0.11925796813690075]
[583749072110.8542 0.245949477304237]
[780044579972.6852 0.23832602714666504]
[2312